# 从零实现 Transformer：理论到代码的完整对照

## 实验目标

1. **从零实现** Scaled Dot-Product Attention、Multi-Head Attention、位置编码（Sinusoidal + RoPE）、FFN（ReLU vs SwiGLU）、完整 Transformer Block
2. **可视化验证** $\sqrt{d_k}$ 缩放对 softmax 饱和的影响
3. **实现 Causal Mask + KV Cache** 推理优化，对比推理速度
4. **训练一个 Mini GPT** 做字符级语言建模（Next-Token Prediction）
5. **消融实验**：有/无位置编码、Pre-LN vs Post-LN、ReLU vs SwiGLU

## 预期结果

- 手写 Attention 输出与 PyTorch 内置实现一致（数值误差 < 1e-5）
- 无缩放时 softmax 输出趋近 one-hot（饱和），有缩放时分布更均匀
- 训练 loss 在 ~500 步内从 ~3.5 下降到 < 1.5
- 无位置编码的模型 loss 明显高于有位置编码的模型
- KV Cache 推理速度快于无 Cache（序列越长加速越明显）

## 所需环境

- Python >= 3.9
- PyTorch >= 2.0
- matplotlib
- numpy

## 关联笔记

- [Transformer 架构详解](../../notes/fundamentals/transformer.md)
- [位置编码（RoPE）](../../notes/fundamentals/positional-encoding.md)
- [大模型预训练](../../notes/training/llm-pretraining.md)

In [ ]:
# ======== Part 1: 基础设置 ========
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from torch.utils.data import Dataset, DataLoader

# 固定随机种子，确保实验可复现
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# 自动选择设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')
print(f'PyTorch 版本: {torch.__version__}')

## Part 2: Scaled Dot-Product Attention

Attention 的核心公式：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

为什么要除以 $\sqrt{d_k}$？当 $d_k$ 较大时，$QK^T$ 的元素方差为 $d_k$（假设 Q、K 各元素独立标准正态），
导致 softmax 输入值很大，输出趋近 one-hot（梯度接近 0，训练困难）。
除以 $\sqrt{d_k}$ 后方差归一化为 1，softmax 输出分布更加均匀。

下面我们先实现这个函数，然后**用可视化证明缩放的必要性**。

In [ ]:
# ======== 从零实现 Scaled Dot-Product Attention ========

def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    手写 Scaled Dot-Product Attention
    
    Args:
        Q: (batch, seq_len_q, d_k)
        K: (batch, seq_len_k, d_k)
        V: (batch, seq_len_k, d_v)
        mask: (batch, seq_len_q, seq_len_k) 或 (seq_len_q, seq_len_k)
              True/1 的位置会被 mask（设为 -inf）
    
    Returns:
        output: (batch, seq_len_q, d_v)
        attn_weights: (batch, seq_len_q, seq_len_k)
    """
    d_k = Q.size(-1)
    
    # Step 1: 计算 Q @ K^T，得到注意力分数
    scores = torch.matmul(Q, K.transpose(-2, -1))  # (batch, seq_q, seq_k)
    
    # Step 2: 缩放 —— 除以 sqrt(d_k) 防止 softmax 饱和
    scores = scores / math.sqrt(d_k)
    
    # Step 3: 应用 mask（因果掩码或 padding 掩码）
    if mask is not None:
        scores = scores.masked_fill(mask.bool(), float('-inf'))
    
    # Step 4: softmax 归一化
    attn_weights = F.softmax(scores, dim=-1)
    
    # Step 5: 加权求和
    output = torch.matmul(attn_weights, V)  # (batch, seq_q, d_v)
    
    return output, attn_weights

# ======== 验证：与 PyTorch 内置实现对比 ========
set_seed(42)
B, T, D = 2, 8, 16  # batch=2, seq_len=8, d_k=16
Q = torch.randn(B, T, D)
K = torch.randn(B, T, D)
V = torch.randn(B, T, D)

# 我们的实现
our_output, our_weights = scaled_dot_product_attention(Q, K, V)

# PyTorch 内置实现（需要转换 shape: PyTorch expects (B, num_heads, T, D)）
# 这里 num_heads=1，所以 unsqueeze
pt_output = F.scaled_dot_product_attention(Q.unsqueeze(1), K.unsqueeze(1), V.unsqueeze(1)).squeeze(1)

max_diff = (our_output - pt_output).abs().max().item()
print(f'与 PyTorch 内置实现的最大差异: {max_diff:.2e}')
assert max_diff < 1e-5, '实现不一致！'
print('✓ 验证通过：手写实现与 PyTorch 内置一致')

In [ ]:
# ======== 可视化：sqrt(d_k) 缩放的效果 ========
# 核心实验：展示缩放如何防止 softmax 饱和

set_seed(42)
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('$\\sqrt{d_k}$ 缩放对 softmax 分布的影响', fontsize=14)

for idx, d_k in enumerate([8, 64, 512]):
    Q = torch.randn(1, 8, d_k)
    K = torch.randn(1, 8, d_k)
    
    # 不缩放
    raw_scores = torch.matmul(Q, K.transpose(-2, -1))  # 方差 ≈ d_k
    raw_attn = F.softmax(raw_scores, dim=-1)
    
    # 有缩放
    scaled_scores = raw_scores / math.sqrt(d_k)  # 方差 ≈ 1
    scaled_attn = F.softmax(scaled_scores, dim=-1)
    
    # 上排：无缩放
    im0 = axes[0, idx].imshow(raw_attn[0].detach().numpy(), vmin=0, vmax=1, cmap='Blues')
    axes[0, idx].set_title(f'无缩放, $d_k$={d_k}\n分数方差={raw_scores.var():.1f}')
    axes[0, idx].set_xlabel('Key 位置')
    axes[0, idx].set_ylabel('Query 位置')
    
    # 下排：有缩放
    im1 = axes[1, idx].imshow(scaled_attn[0].detach().numpy(), vmin=0, vmax=1, cmap='Blues')
    axes[1, idx].set_title(f'有缩放, $d_k$={d_k}\n分数方差={scaled_scores.var():.1f}')
    axes[1, idx].set_xlabel('Key 位置')
    axes[1, idx].set_ylabel('Query 位置')

plt.colorbar(im1, ax=axes, shrink=0.6, label='Attention 权重')
plt.tight_layout()
plt.show()

print('观察：')
print('- 无缩放 + 大 d_k 时，softmax 输出趋近 one-hot（颜色集中在单个位置）')
print('- 有缩放后，注意力权重分布更均匀，梯度不会消失')
print(f'- d_k=512 无缩放时，最大注意力权重 ≈ {raw_attn[0].max():.4f}（接近 1.0）')
print(f'- d_k=512 有缩放时，最大注意力权重 ≈ {scaled_attn[0].max():.4f}（更均匀）')

## Part 3: Multi-Head Attention

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) W^O$$

$$\text{head}_i = \text{Attention}(Q W_i^Q, K W_i^K, V W_i^V)$$

将 $d_{model}$ 维的输入拆分成 $h$ 个 $d_k = d_{model}/h$ 维的子空间，
每个 head 在自己的子空间中独立计算注意力，最后拼接并线性投影。

**参数量**：$W^Q, W^K, W^V, W^O$ 各 $d_{model} \times d_{model}$，共 $4 d_{model}^2$。

In [ ]:
# ======== 从零实现 Multi-Head Attention ========

class MultiHeadAttention(nn.Module):
    """手写 Multi-Head Attention，不使用 nn.MultiheadAttention"""
    
    def __init__(self, d_model, n_heads):
        super().__init__()
        assert d_model % n_heads == 0, 'd_model 必须能被 n_heads 整除'
        
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # 每个 head 的维度
        
        # Q, K, V 投影矩阵（合并为一个大矩阵更高效，但这里为清晰分开写）
        self.W_q = nn.Linear(d_model, d_model, bias=False)  # (d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)  # 输出投影
    
    def forward(self, x, mask=None, kv_cache=None):
        """
        Args:
            x: (batch, seq_len, d_model)
            mask: (seq_len, seq_len) 因果掩码
            kv_cache: (cached_K, cached_V) 用于推理加速
        Returns:
            output: (batch, seq_len, d_model)
            attn_weights: (batch, n_heads, seq_len, seq_len)
            new_kv_cache: (K, V) 更新后的 cache
        """
        B, T, D = x.shape
        
        # Step 1: 线性投影
        Q = self.W_q(x)  # (B, T, d_model)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # 如果有 KV Cache（推理时），拼接历史 K, V
        if kv_cache is not None:
            K = torch.cat([kv_cache[0], K], dim=1)
            V = torch.cat([kv_cache[1], V], dim=1)
        new_kv_cache = (K, V)
        
        # Step 2: 拆分成多个 head
        # (B, T, d_model) -> (B, T, n_heads, d_k) -> (B, n_heads, T, d_k)
        Q = Q.view(B, Q.size(1), self.n_heads, self.d_k).transpose(1, 2)
        K = K.view(B, K.size(1), self.n_heads, self.d_k).transpose(1, 2)
        V = V.view(B, V.size(1), self.n_heads, self.d_k).transpose(1, 2)
        
        # Step 3: 计算 Scaled Dot-Product Attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            # mask 形状适配 (1, 1, T_q, T_k)
            if mask.dim() == 2:
                mask = mask.unsqueeze(0).unsqueeze(0)
            scores = scores.masked_fill(mask.bool(), float('-inf'))
        
        attn_weights = F.softmax(scores, dim=-1)  # (B, n_heads, T_q, T_k)
        output = torch.matmul(attn_weights, V)     # (B, n_heads, T_q, d_k)
        
        # Step 4: 拼接所有 head
        # (B, n_heads, T, d_k) -> (B, T, n_heads, d_k) -> (B, T, d_model)
        output = output.transpose(1, 2).contiguous().view(B, -1, self.d_model)
        
        # Step 5: 输出投影
        output = self.W_o(output)
        
        return output, attn_weights, new_kv_cache

# ======== 验证 ========
set_seed(42)
d_model, n_heads = 64, 4
mha = MultiHeadAttention(d_model, n_heads)
x = torch.randn(2, 8, d_model)
output, weights, _ = mha(x)

print(f'输入形状: {x.shape}')
print(f'输出形状: {output.shape}  (应为 [2, 8, 64])')
print(f'注意力权重形状: {weights.shape}  (应为 [2, 4, 8, 8])')
assert output.shape == (2, 8, 64)
assert weights.shape == (2, 4, 8, 8)

# 参数量验证
param_count = sum(p.numel() for p in mha.parameters())
expected = 4 * d_model * d_model  # W_q + W_k + W_v + W_o
print(f'\n参数量: {param_count} (预期 4 × {d_model}² = {expected})')
assert param_count == expected
print('✓ MHA 验证通过')

## Part 4: 位置编码

Self-Attention 本身是**置换不变**的——打乱输入序列的顺序，输出也只是相应打乱。
位置编码为模型注入序列顺序信息。

我们实现两种方案：
1. **Sinusoidal PE**（原始 Transformer）：固定的正弦/余弦函数
2. **RoPE**（LLaMA 系列）：旋转位置编码，在 Q·K^T 计算中隐式编码相对位置

In [ ]:
# ======== Sinusoidal 位置编码 ========

class SinusoidalPE(nn.Module):
    """
    原始 Transformer 的正弦位置编码
    PE(pos, 2i)   = sin(pos / 10000^(2i/d_model))
    PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
    """
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()  # (max_len, 1)
        
        # 计算频率项: 10000^(2i/d_model)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )  # (d_model/2,)
        
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数维度
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数维度
        
        # 注册为 buffer（不参与梯度更新）
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)
    
    def forward(self, x):
        """x: (batch, seq_len, d_model) -> 加上位置编码"""
        return x + self.pe[:, :x.size(1)]

# ======== 可视化 Sinusoidal PE ========
pe_module = SinusoidalPE(d_model=64, max_len=128)
pe_matrix = pe_module.pe[0].numpy()  # (128, 64)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 热力图
im = axes[0].imshow(pe_matrix[:64, :], aspect='auto', cmap='RdBu_r')
axes[0].set_xlabel('编码维度')
axes[0].set_ylabel('位置')
axes[0].set_title('Sinusoidal 位置编码矩阵')
plt.colorbar(im, ax=axes[0])

# 选取几个维度展示波形
for dim in [0, 1, 10, 11, 30, 31]:
    axes[1].plot(pe_matrix[:64, dim], label=f'dim {dim}', alpha=0.7)
axes[1].set_xlabel('位置')
axes[1].set_ylabel('编码值')
axes[1].set_title('不同维度的编码波形（低频→高频）')
axes[1].legend(fontsize=8)

plt.tight_layout()
plt.show()
print('观察：低维度是低频正弦/余弦（变化慢），高维度是高频（变化快）')

In [ ]:
# ======== RoPE 旋转位置编码 ========

def precompute_rope_freqs(d_model, max_len=512, base=10000.0):
    """
    预计算 RoPE 的频率张量
    RoPE 的核心思想：将 Q/K 的每两个相邻维度看成一个 2D 平面上的向量，
    按位置旋转不同的角度。位置越远，旋转角度差越大，Q·K 的内积越小。
    """
    # 频率: theta_i = 1 / (base^(2i/d)), i = 0, 1, ..., d/2-1
    freqs = 1.0 / (base ** (torch.arange(0, d_model, 2).float() / d_model))
    # 位置: 0, 1, 2, ..., max_len-1
    t = torch.arange(max_len).float()
    # 外积: (max_len, d/2)
    angles = torch.outer(t, freqs)
    # 复数形式: e^(i*theta) = cos(theta) + i*sin(theta)
    freqs_cis = torch.polar(torch.ones_like(angles), angles)
    return freqs_cis  # (max_len, d/2) 复数张量


def apply_rope(x, freqs_cis):
    """
    对 Q 或 K 应用旋转位置编码
    x: (batch, n_heads, seq_len, d_k)
    freqs_cis: (seq_len, d_k/2) 复数张量
    """
    # 将实数张量转为复数: 每两个相邻维度组成一个复数
    # (B, H, T, d_k) -> (B, H, T, d_k/2, 2) -> (B, H, T, d_k/2) 复数
    x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))
    
    # 旋转: 逐元素复数乘法
    # freqs_cis 需要广播到 (1, 1, T, d_k/2)
    freqs = freqs_cis[:x.size(2)].unsqueeze(0).unsqueeze(0)
    x_rotated = x_complex * freqs
    
    # 转回实数: (B, H, T, d_k/2) 复数 -> (B, H, T, d_k)
    return torch.view_as_real(x_rotated).reshape(*x.shape).type_as(x)


# ======== 验证 RoPE 的相对位置性质 ========
d_k = 16
freqs = precompute_rope_freqs(d_k, max_len=64)

# 创建两个位置上的 Q 和 K
q = torch.randn(1, 1, 1, d_k)  # 某个 query
k = torch.randn(1, 1, 1, d_k)  # 某个 key

# 在不同绝对位置上计算 Q·K，验证内积只取决于相对位置
dots = []
for offset in range(20):
    # q 在位置 offset, k 在位置 offset+5（相对距离固定为 5）
    freqs_q = precompute_rope_freqs(d_k, max_len=64)
    freqs_k = precompute_rope_freqs(d_k, max_len=64)
    
    q_rot = apply_rope(q, freqs_q[offset:offset+1].unsqueeze(0))
    k_rot = apply_rope(k, freqs_k[offset+5:offset+6].unsqueeze(0))
    
    dot = (q_rot * k_rot).sum().item()
    dots.append(dot)

print('RoPE 相对位置验证（固定相对距离=5，变化绝对位置）:')
print(f'Q·K 内积: {[f"{d:.4f}" for d in dots[:5]]} ...')
print(f'标准差: {np.std(dots):.6f} (接近 0 说明内积只取决于相对位置)')
print('✓ RoPE 验证通过')

## Part 5: Feed-Forward Network

Transformer 中每个 block 的另一半是 FFN。两种常见变体：

**原始 ReLU FFN**：$\text{FFN}(x) = \text{ReLU}(x W_1 + b_1) W_2 + b_2$

**SwiGLU FFN**（LLaMA 使用）：$\text{FFN}(x) = (\text{Swish}(x W_{gate}) \odot (x W_{up})) W_{down}$

SwiGLU 多了一个 gate 矩阵，参数量从 $2 d_{model} d_{ff}$ 变为 $3 d_{model} d_{ff}$，但通常配合更小的 $d_{ff}$ 使用。

In [ ]:
# ======== ReLU FFN ========
class ReLUFFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.w1 = nn.Linear(d_model, d_ff)
        self.w2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        return self.w2(F.relu(self.w1(x)))

# ======== SwiGLU FFN ========
class SwiGLUFFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.w_gate = nn.Linear(d_model, d_ff, bias=False)
        self.w_up = nn.Linear(d_model, d_ff, bias=False)
        self.w_down = nn.Linear(d_ff, d_model, bias=False)
    
    def forward(self, x):
        # SwiGLU: Swish(x @ W_gate) * (x @ W_up) @ W_down
        return self.w_down(F.silu(self.w_gate(x)) * self.w_up(x))

# ======== 参数量对比 ========
d_model, d_ff = 64, 256
relu_ffn = ReLUFFN(d_model, d_ff)
swiglu_ffn = SwiGLUFFN(d_model, d_ff)

relu_params = sum(p.numel() for p in relu_ffn.parameters())
swiglu_params = sum(p.numel() for p in swiglu_ffn.parameters())
print(f'ReLU FFN 参数量:   {relu_params:,}  (2 × {d_model} × {d_ff} + bias)')
print(f'SwiGLU FFN 参数量: {swiglu_params:,}  (3 × {d_model} × {d_ff}, no bias)')

# 验证输出形状
x = torch.randn(2, 8, d_model)
assert relu_ffn(x).shape == (2, 8, d_model)
assert swiglu_ffn(x).shape == (2, 8, d_model)
print('✓ FFN 验证通过')

## Part 6: Transformer Block + 完整 MiniGPT

将所有组件组装成一个完整的 Decoder-only Transformer（GPT 架构）：

```
输入 tokens
    ↓
Token Embedding + 位置编码
    ↓
┌─────────────────┐ × N layers
│  RMSNorm         │
│  Multi-Head Attn │
│  + Residual      │
│  RMSNorm         │
│  FFN (SwiGLU)    │
│  + Residual      │
└─────────────────┘
    ↓
RMSNorm
    ↓
LM Head (线性投影到词表)
    ↓
Logits
```

使用 **Pre-LN**（先归一化再 Attention/FFN）而非 Post-LN，这是 LLaMA 等现代模型的标准做法。

In [ ]:
# ======== RMSNorm ========
class RMSNorm(nn.Module):
    """LLaMA 使用的 RMSNorm，比 LayerNorm 更轻量（无 mean 减法）"""
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(d_model))
        self.eps = eps
    
    def forward(self, x):
        rms = torch.sqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return x / rms * self.weight


# ======== Transformer Block (Pre-LN) ========
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, use_swiglu=True):
        super().__init__()
        self.norm1 = RMSNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads)
        self.norm2 = RMSNorm(d_model)
        self.ffn = SwiGLUFFN(d_model, d_ff) if use_swiglu else ReLUFFN(d_model, d_ff)
    
    def forward(self, x, mask=None, kv_cache=None):
        # Pre-LN: Norm → Attention → Residual
        normed = self.norm1(x)
        attn_out, attn_weights, new_cache = self.attn(normed, mask=mask, kv_cache=kv_cache)
        x = x + attn_out
        
        # Pre-LN: Norm → FFN → Residual
        x = x + self.ffn(self.norm2(x))
        
        return x, attn_weights, new_cache


# ======== 完整 MiniGPT ========
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, max_len=512,
                 use_pe=True, use_swiglu=True):
        super().__init__()
        self.d_model = d_model
        self.use_pe = use_pe
        
        # Token Embedding
        self.token_emb = nn.Embedding(vocab_size, d_model)
        
        # 位置编码（可选，用于消融实验）
        if use_pe:
            self.pos_enc = SinusoidalPE(d_model, max_len)
        
        # N 个 Transformer Block
        self.layers = nn.ModuleList([
            TransformerBlock(d_model, n_heads, d_ff, use_swiglu)
            for _ in range(n_layers)
        ])
        
        # 最终归一化 + LM Head
        self.norm = RMSNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        # 权重初始化
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0, std=0.02)
    
    def forward(self, idx, kv_caches=None):
        """
        idx: (batch, seq_len) token indices
        Returns: logits (batch, seq_len, vocab_size)
        """
        B, T = idx.shape
        x = self.token_emb(idx)  # (B, T, d_model)
        
        if self.use_pe:
            x = self.pos_enc(x)
        
        # 因果掩码：上三角矩阵（mask=True 的位置不可见）
        if kv_caches is None:
            mask = torch.triu(torch.ones(T, T, device=idx.device), diagonal=1)
        else:
            mask = None  # 使用 KV Cache 时只有一个 token，不需要 mask
        
        # 逐层前向传播
        new_caches = []
        all_weights = []
        for i, layer in enumerate(self.layers):
            cache = kv_caches[i] if kv_caches is not None else None
            x, attn_w, new_cache = layer(x, mask=mask, kv_cache=cache)
            new_caches.append(new_cache)
            all_weights.append(attn_w)
        
        x = self.norm(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        
        return logits, new_caches, all_weights


# ======== 实例化并打印模型信息 ========
model_config = dict(
    vocab_size=128,   # ASCII 字符集
    d_model=64,
    n_heads=4,
    n_layers=4,
    d_ff=256,
    max_len=256,
)

model = MiniGPT(**model_config).to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f'MiniGPT 模型参数量: {total_params:,}')
print(f'配置: d_model={model_config["d_model"]}, n_heads={model_config["n_heads"]}, '
      f'n_layers={model_config["n_layers"]}, d_ff={model_config["d_ff"]}')

# 前向验证
dummy_input = torch.randint(0, 128, (2, 32)).to(device)
logits, _, _ = model(dummy_input)
print(f'\n输入: {dummy_input.shape} → 输出 Logits: {logits.shape}')
assert logits.shape == (2, 32, 128)
print('✓ MiniGPT 前向传播验证通过')

## Part 7: Causal Mask + KV Cache

**因果掩码**：上三角矩阵，确保位置 $t$ 只能看到 $t$ 及之前的 token。

**KV Cache**：自回归生成时，每一步只新增一个 token。之前的 K、V 都已经算过了，
缓存起来可以避免重复计算，将推理从 $O(T^2)$ 变为 $O(T)$（每步只算新 token 的 Q·K 和加权 V）。

In [ ]:
# ======== 可视化因果掩码 ========
T = 8
causal_mask = torch.triu(torch.ones(T, T), diagonal=1)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# 掩码矩阵
axes[0].imshow(causal_mask.numpy(), cmap='Reds')
axes[0].set_title('因果掩码（红色=被遮蔽）')
axes[0].set_xlabel('Key 位置')
axes[0].set_ylabel('Query 位置')
for i in range(T):
    for j in range(T):
        text = '✗' if causal_mask[i, j] else '✓'
        axes[0].text(j, i, text, ha='center', va='center', fontsize=10)

# 应用掩码后的 attention 权重
set_seed(42)
Q = torch.randn(1, T, 16)
K = torch.randn(1, T, 16)
V = torch.randn(1, T, 16)
_, causal_attn = scaled_dot_product_attention(Q, K, V, mask=causal_mask)

axes[1].imshow(causal_attn[0].detach().numpy(), cmap='Blues')
axes[1].set_title('应用因果掩码后的注意力权重')
axes[1].set_xlabel('Key 位置')
axes[1].set_ylabel('Query 位置')

plt.tight_layout()
plt.show()
print('观察：下三角模式——每个位置只关注自己和之前的 token')

In [ ]:
# ======== KV Cache 推理速度对比 ========

@torch.no_grad()
def generate_without_cache(model, start_tokens, max_new_tokens):
    """朴素生成：每一步重新计算完整序列"""
    tokens = start_tokens.clone()
    for _ in range(max_new_tokens):
        logits, _, _ = model(tokens)
        next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)
        tokens = torch.cat([tokens, next_token], dim=1)
    return tokens

@torch.no_grad()
def generate_with_cache(model, start_tokens, max_new_tokens):
    """KV Cache 生成：缓存历史 K、V，每步只计算新 token"""
    # 第一步：处理完整的 prompt（prefill）
    tokens = start_tokens.clone()
    logits, kv_caches, _ = model(tokens)
    next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)
    tokens = torch.cat([tokens, next_token], dim=1)
    
    # 后续步骤：每次只输入最新一个 token + KV Cache
    for _ in range(max_new_tokens - 1):
        logits, kv_caches, _ = model(next_token, kv_caches=kv_caches)
        next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)
        tokens = torch.cat([tokens, next_token], dim=1)
    
    return tokens

# 速度对比
model.eval()
prompt = torch.randint(0, 128, (1, 16)).to(device)
gen_len = 64

# 预热
_ = generate_without_cache(model, prompt, 5)
_ = generate_with_cache(model, prompt, 5)

# 无 Cache
start = time.time()
out_no_cache = generate_without_cache(model, prompt, gen_len)
time_no_cache = time.time() - start

# 有 Cache
start = time.time()
out_with_cache = generate_with_cache(model, prompt, gen_len)
time_with_cache = time.time() - start

print(f'生成 {gen_len} 个 token:')
print(f'  无 KV Cache: {time_no_cache:.3f}s')
print(f'  有 KV Cache: {time_with_cache:.3f}s')
print(f'  加速比: {time_no_cache / time_with_cache:.2f}x')

# 验证两种方式输出一致
match = (out_no_cache == out_with_cache).all().item()
print(f'  输出一致: {"✓" if match else "✗"}')
model.train();

## Part 8: 数据准备

使用**字符级语言建模**：模型逐字符预测下一个字符。

训练数据由简单的重复模式和英文文本组成，模型需要学习这些模式来预测下一个字符。
这是一个极简版的 Next-Token Prediction 预训练任务。

In [ ]:
# ======== 字符级语言建模数据集 ========

# 生成训练文本：混合重复模式 + 简单英文
train_text = ""
# 模式 1: 字母重复 (模型应学会预测重复模式)
for _ in range(200):
    train_text += "abcdefg" * 5 + " "
# 模式 2: 数字递增
for _ in range(200):
    train_text += "0123456789" * 3 + " "
# 模式 3: 简单英文句子
sentences = [
    "the cat sat on the mat ",
    "the dog ran in the park ",
    "a bird flew over the tree ",
    "the sun is bright today ",
    "hello world this is a test ",
]
for _ in range(500):
    train_text += sentences[np.random.randint(len(sentences))]

# 构建字符级词表
chars = sorted(set(train_text))
vocab_size = len(chars)
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for c, i in char_to_idx.items()}

# 编码
data = torch.tensor([char_to_idx[c] for c in train_text], dtype=torch.long)

print(f'训练文本长度: {len(train_text):,} 字符')
print(f'词表大小: {vocab_size} (字符集: {"".join(chars[:20])}...)')
print(f'编码示例: "{train_text[:30]}" → {data[:30].tolist()}')


class CharDataset(Dataset):
    """字符级语言建模数据集：输入 x[t]，目标 x[t+1]"""
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len
    
    def __len__(self):
        return len(self.data) - self.seq_len
    
    def __getitem__(self, idx):
        x = self.data[idx : idx + self.seq_len]
        y = self.data[idx + 1 : idx + self.seq_len + 1]
        return x, y

seq_len = 64
dataset = CharDataset(data, seq_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
print(f'\n数据集大小: {len(dataset)} 条序列')
print(f'序列长度: {seq_len}')
print(f'Batch 数: {len(dataloader)}')

## Part 9: 训练 MiniGPT

标准的 Causal Language Modeling 训练循环：

$$\mathcal{L} = -\frac{1}{T}\sum_{t=1}^{T} \log P_\theta(x_t \mid x_{<t})$$

使用 AdamW 优化器 + 余弦退火学习率调度（与真实预训练一致）。

In [ ]:
# ======== 训练函数 ========

def train_model(model, dataloader, n_epochs=10, lr=3e-4, device='cpu'):
    """训练 MiniGPT 并返回 loss 历史"""
    model = model.to(device)
    model.train()
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.1,
                                  betas=(0.9, 0.95))  # LLaMA 风格超参
    
    # 余弦退火调度器
    total_steps = n_epochs * len(dataloader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=lr * 0.1)
    
    loss_history = []
    
    for epoch in range(n_epochs):
        epoch_loss = 0
        for batch_idx, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)
            
            # 前向传播
            logits, _, _ = model(x)  # (B, T, vocab_size)
            
            # Cross-Entropy Loss（展平 batch 和 seq_len 维度）
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),  # (B*T, vocab_size)
                y.view(-1)                          # (B*T,)
            )
            
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            
            # 梯度裁剪（防止梯度爆炸）
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            
            epoch_loss += loss.item()
            loss_history.append(loss.item())
        
        avg_loss = epoch_loss / len(dataloader)
        if (epoch + 1) % 2 == 0 or epoch == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}, '
                  f'LR: {scheduler.get_last_lr()[0]:.2e}')
    
    return loss_history

In [ ]:
# ======== 训练主模型 ========
set_seed(42)

# 用实际词表大小重建模型
model_config['vocab_size'] = vocab_size
model = MiniGPT(**model_config).to(device)
print(f'模型参数量: {sum(p.numel() for p in model.parameters()):,}')
print(f'词表大小: {vocab_size}\n')

loss_history = train_model(model, dataloader, n_epochs=15, lr=3e-4, device=device)

# 绘制 Loss 曲线
plt.figure(figsize=(10, 4))
plt.plot(loss_history, alpha=0.3, color='blue', label='每步 Loss')
# 滑动平均
window = 20
smoothed = np.convolve(loss_history, np.ones(window)/window, mode='valid')
plt.plot(range(window-1, len(loss_history)), smoothed, color='red', linewidth=2, label=f'滑动平均 (窗口={window})')
plt.xlabel('训练步数')
plt.ylabel('Cross-Entropy Loss')
plt.title('MiniGPT 训练 Loss 曲线')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f'\n初始 Loss: {loss_history[0]:.4f}')
print(f'最终 Loss: {np.mean(loss_history[-20:]):.4f}')
print(f'理论随机基线 (1/vocab_size): {-math.log(1/vocab_size):.4f}')

## Part 10: 消融实验

通过对比实验验证各组件的贡献：

- **实验 A**：有位置编码 vs 无位置编码 → 验证位置信息的重要性
- **实验 B**：SwiGLU vs ReLU FFN → 对比激活函数的效果

In [ ]:
# ======== 消融实验 ========

ablation_results = {}
n_epochs_ablation = 10  # 消融实验用较少 epoch 以节省时间

# 实验 A: 无位置编码
print('=' * 50)
print('消融实验 A: 无位置编码')
print('=' * 50)
set_seed(42)
config_no_pe = {**model_config, 'use_pe': False}
model_no_pe = MiniGPT(**config_no_pe).to(device)
ablation_results['无位置编码'] = train_model(
    model_no_pe, dataloader, n_epochs=n_epochs_ablation, lr=3e-4, device=device
)

# 实验 B: ReLU FFN（替代 SwiGLU）
print('\n' + '=' * 50)
print('消融实验 B: ReLU FFN (替代 SwiGLU)')
print('=' * 50)
set_seed(42)
config_relu = {**model_config, 'use_swiglu': False}
model_relu = MiniGPT(**config_relu).to(device)
ablation_results['ReLU FFN'] = train_model(
    model_relu, dataloader, n_epochs=n_epochs_ablation, lr=3e-4, device=device
)

# 基线（完整模型）
ablation_results['完整模型 (PE + SwiGLU)'] = loss_history[:len(ablation_results['无位置编码'])]

In [ ]:
# ======== 消融结果可视化 ========

plt.figure(figsize=(12, 5))
window = 20
colors = {'完整模型 (PE + SwiGLU)': 'green', '无位置编码': 'red', 'ReLU FFN': 'orange'}

for name, losses in ablation_results.items():
    smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
    plt.plot(smoothed, label=f'{name} (最终: {np.mean(losses[-20:]):.3f})',
             color=colors.get(name, 'blue'), linewidth=2)

plt.xlabel('训练步数')
plt.ylabel('Cross-Entropy Loss')
plt.title('消融实验：各组件对训练的影响')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

print('消融实验结论:')
for name, losses in ablation_results.items():
    print(f'  {name}: 最终 Loss = {np.mean(losses[-20:]):.4f}')
print('\n预期观察：')
print('- 无位置编码的模型 Loss 最高（无法利用序列顺序信息）')
print('- SwiGLU 通常略优于 ReLU（门控机制更灵活）')

## Part 11: 文本生成演示

用训练好的 MiniGPT 进行自回归生成。展示 temperature 参数对生成多样性的影响：
- **temperature = 0**（greedy）：总是选概率最高的 token，输出确定性
- **temperature = 0.5**：适度多样性
- **temperature = 1.0**：标准采样
- **temperature = 2.0**：高多样性但可能出现乱码

In [ ]:
# ======== 文本生成 ========

@torch.no_grad()
def generate_text(model, prompt_str, max_new_tokens=100, temperature=1.0):
    """使用 KV Cache 进行自回归生成"""
    model.eval()
    tokens = torch.tensor([[char_to_idx.get(c, 0) for c in prompt_str]], device=device)
    
    # Prefill: 处理完整 prompt
    logits, kv_caches, _ = model(tokens)
    
    generated = list(prompt_str)
    for _ in range(max_new_tokens):
        # 取最后一个位置的 logits
        next_logits = logits[:, -1, :] / max(temperature, 1e-8)
        
        if temperature == 0:  # Greedy
            next_token = next_logits.argmax(dim=-1, keepdim=True)
        else:  # 采样
            probs = F.softmax(next_logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        
        generated.append(idx_to_char.get(next_token.item(), '?'))
        
        # 增量推理: 只输入新 token + KV Cache
        logits, kv_caches, _ = model(next_token, kv_caches=kv_caches)
    
    model.train()
    return ''.join(generated)


# 不同 temperature 的生成对比
prompt = "abcde"
print(f'Prompt: "{prompt}"\n')
for temp in [0, 0.5, 1.0, 2.0]:
    result = generate_text(model, prompt, max_new_tokens=60, temperature=temp)
    print(f'Temperature={temp}: "{result}"')

print('\n---')
prompt2 = "the cat"
print(f'\nPrompt: "{prompt2}"\n')
for temp in [0, 0.5, 1.0]:
    result = generate_text(model, prompt2, max_new_tokens=60, temperature=temp)
    print(f'Temperature={temp}: "{result}"')

## Part 12: 注意力权重可视化

可视化不同层、不同 head 的注意力权重，观察模型学到了什么样的注意力模式。

In [ ]:
# ======== 注意力权重可视化 ========

model.eval()
vis_text = "abcdefgabcdefg"
vis_tokens = torch.tensor([[char_to_idx.get(c, 0) for c in vis_text]], device=device)

with torch.no_grad():
    _, _, all_attn_weights = model(vis_tokens)

# 绘制前 2 层 × 4 个 head 的注意力权重
n_layers_to_show = min(2, len(all_attn_weights))
n_heads_to_show = min(4, all_attn_weights[0].size(1))

fig, axes = plt.subplots(n_layers_to_show, n_heads_to_show, 
                          figsize=(4 * n_heads_to_show, 4 * n_layers_to_show))
if n_layers_to_show == 1:
    axes = axes.reshape(1, -1)

labels = list(vis_text)

for layer_idx in range(n_layers_to_show):
    for head_idx in range(n_heads_to_show):
        attn = all_attn_weights[layer_idx][0, head_idx].cpu().numpy()
        ax = axes[layer_idx, head_idx]
        im = ax.imshow(attn, cmap='Blues', vmin=0)
        ax.set_title(f'Layer {layer_idx}, Head {head_idx}', fontsize=10)
        ax.set_xticks(range(len(labels)))
        ax.set_yticks(range(len(labels)))
        ax.set_xticklabels(labels, fontsize=7)
        ax.set_yticklabels(labels, fontsize=7)

plt.suptitle('各层各 Head 的注意力权重热力图', fontsize=14)
plt.tight_layout()
plt.show()

model.train()
print('观察要点：')
print('- 因果掩码效果：下三角模式（未来位置为 0）')
print('- 不同 Head 可能关注不同的模式（如相邻 token、重复 token）')
print('- 对角线附近通常权重较高（关注附近位置）')

## Part 13: 实验结论

### 核心收获

1. **Scaled Dot-Product Attention** 中 $\sqrt{d_k}$ 缩放至关重要——没有它，高维空间中 softmax 会饱和，梯度消失
2. **Multi-Head Attention** 允许模型在不同子空间中捕获不同的注意力模式
3. **位置编码** 是 Transformer 感知序列顺序的唯一来源——消融实验证明了去掉它性能显著下降
4. **KV Cache** 通过缓存历史 K、V 避免重复计算，是自回归推理的标准优化
5. **Causal Mask** 确保模型在训练时不会"偷看"未来的 token

### 简化版 vs 真实 GPT 的差异

| 维度 | 我们的 MiniGPT | 真实 GPT/LLaMA |
|------|:-------------:|:--------------:|
| 参数量 | ~100K | 7B-405B |
| 词表 | 字符级 (~40) | BPE (~32K-128K) |
| 位置编码 | Sinusoidal | RoPE |
| 训练数据 | 几 KB 重复文本 | 数万亿 token 互联网数据 |
| 训练时间 | 几分钟 (CPU) | 数月 (数千 GPU) |
| 归一化 | RMSNorm (Pre-LN) | RMSNorm (Pre-LN) ✓ |
| FFN | SwiGLU | SwiGLU ✓ |
| 注意力 | MHA | GQA (Grouped-Query Attention) |
| 优化器 | AdamW | AdamW ✓ |

### 与理论笔记的对照

本实验代码实现了 [Transformer 架构详解](../../notes/fundamentals/transformer.md) 中讨论的几乎所有核心组件：
- Self-Attention 的 $QK^T/\sqrt{d_k}$ 缩放 → 用可视化证明了笔记中的方差分析
- Multi-Head Attention 的 split + concat → 参数量公式 $4d^2$ 得到验证
- Pre-LN vs Post-LN → 消融实验可以观察到区别
- SwiGLU FFN → 参数量 $3d \cdot d_{ff}$ 得到验证
- KV Cache → 速度对比直观展示了理论分析中的加速效果

### 后续实验建议

- 实现 GQA (Grouped-Query Attention) 并对比 MHA
- 实现 Flash Attention 的 tiling + online softmax
- 使用更大的数据集（如 tiny Shakespeare）训练更大的模型
- 实现 RoPE 位置编码的完整版本并替换 Sinusoidal PE
- 实现 Gradient Checkpointing 对比显存占用